# Import thư viện và nạp dữ liệu

In [12]:
# Import thư viện

from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Tuple

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, RobustScaler

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from dataclasses import dataclass

RANDOM_STATE: int = 42  # Seed để tái lập kết quả qua các lần chạy
np.random.seed(RANDOM_STATE)  # Cố định ngẫu nhiên cho numpy
pd.set_option('display.max_columns', 200)  # Hỗ trợ kiểm tra schema trực quan
pd.set_option('display.width', 200)  # Hỗ trợ hiển thị rộng hơn

print('Đã import thư viện thành công. Sẵn sàng sang bước nạp dữ liệu.')

Đã import thư viện thành công. Sẵn sàng sang bước nạp dữ liệu.


# Nạp dữ liệu (train/test)

## 1) Mục tiêu của bước này
Nhóm nạp hai tập dữ liệu `train_raw` và `test_raw` từ `./data/cleaned/` để làm đầu vào cho các thí nghiệm và các bước kỹ thuật đặc trưng tiếp theo.

## 2) Cơ sở lý thuyết
Việc nạp và kiểm tra dữ liệu ngay sau bước import giúp nhóm xác nhận: (i) schema có nhất quán giữa train và test hay không, (ii) các biến số thô phục vụ thí nghiệm baseline (OLS) có sẵn hay không, và (iii) biến mục tiêu `vote_average` có đầy đủ hay không. Điều này giảm rủi ro sai lệch khi nhóm đánh giá mối quan hệ tuyến tính ban đầu.

## 3) Giả định (Assumptions)
Trước khi xây dựng giả thuyết Feature Engineering chính thức, nhóm đặt các giả định tối thiểu để chạy thí nghiệm pilot:
- `vote_average` trong train không thiếu.
- Các biến số thô dùng cho OLS (ví dụ `budget`, `revenue`, `popularity`, `runtime`) có thể có NaN và sẽ được điền nhanh bằng median *chỉ để chạy baseline*.
- Các cột văn bản/categorical có thể chứa NaN và sẽ xử lý ở các bước sau.

## 4) Thực hiện
- Đọc `train_raw.csv` và `test_raw.csv`.
- Kiểm tra nhanh: kích thước, `head()`, `info()`, và đối chiếu danh sách cột giữa train và test.
- Xác nhận schema nhất quán để đảm bảo các bước biến đổi sau áp dụng đồng bộ.

## 5) Kết luận sơ bộ
Dựa trên output của Cell 4 (nạp dữ liệu), nhóm ghi nhận:
- Kích thước dữ liệu: train có 8.000 quan sát và test có 2.000 quan sát; cả hai tập đều gồm 21 cột.
- Tính nhất quán schema: danh sách cột giữa train và test trùng khớp.
- Thiếu dữ liệu tập trung ở biến văn bản: `overview` và đặc biệt `tagline`. Nhóm sẽ giữ NaN ở các biến này để xử lý robust ở bước Pipeline.
- Dữ liệu có 6 biến số `float64`, 1 biến số `int64` và 14 biến `object`, phù hợp để thực hiện baseline OLS trên nhóm biến số trước khi thiết kế biến đổi đặc trưng.

In [ ]:
# Nạp dữ liệu (giữ nguyên NaN; chỉ kiểm tra các cột bắt buộc tối thiểu)

# Xác định project root để chạy ổn định (dù mở notebook từ thư mục nào)
project_root = Path.cwd()
if project_root.name.lower() == 'notebooks':
    project_root = project_root.parent

train_path = project_root / 'data' / 'cleaned' / 'train_raw.csv'
test_path = project_root / 'data' / 'cleaned' / 'test_raw.csv'

train_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

print('Train shape:', train_raw.shape)
print('Test shape :', test_raw.shape)

print('\n===== TRAIN (head) =====')
display(train_raw.head())
print('\n===== TEST (head) =====')
display(test_raw.head())

print('\n===== TRAIN (info) =====')
train_buf = io.StringIO()
train_raw.info(buf=train_buf)
print(train_buf.getvalue())

print('\n===== TEST (info) =====')
test_buf = io.StringIO()
test_raw.info(buf=test_buf)
print(test_buf.getvalue())

# Kiểm tra nhất quán schema giữa train và test
train_cols = set(train_raw.columns)
test_cols = set(test_raw.columns)
print('\\nSố cột train:', len(train_cols))
print('Số cột test :', len(test_cols))
print('Cột chỉ có ở train:', sorted(list(train_cols - test_cols))[:50])
print('Cột chỉ có ở test :', sorted(list(test_cols - train_cols))[:50])

# Kiểm tra điều kiện tối thiểu cho các giả thuyết (budget/revenue) và biến mục tiêu
required_train_cols = ['budget', 'revenue', 'vote_average']
missing_required_train_cols = [c for c in required_train_cols if c not in train_raw.columns]
if len(missing_required_train_cols) > 0:
    raise ValueError(f"Train thiếu các cột bắt buộc: {missing_required_train_cols}")

required_test_cols = ['budget', 'revenue']
missing_required_test_cols = [c for c in required_test_cols if c not in test_raw.columns]
if len(missing_required_test_cols) > 0:
    raise ValueError(f"Test thiếu các cột bắt buộc: {missing_required_test_cols}")

missing_counts = train_raw[required_train_cols].isna().sum()
print('\\nMissing (train) cho các cột bắt buộc:')
print(missing_counts.to_string())

if int(missing_counts.sum()) != 0:
    raise ValueError('Train có NaN ở budget/revenue/vote_average; cần xử lý trước khi Feature Engineering.')

Train shape: (8000, 21)
Test shape : (2000, 21)

===== TRAIN (head) =====


,budget,cast_top5,certification_US,collection_id,directors,genres,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_count,vote_average
0,2.000000e+06,"Ethan Hawke,Julie Delpy,Wiley Wiggins,Bill Wis...",NR,-1.0,Richard Linklater,"Animation,Drama,Fantasy","dreams,philosophy,parallel world,existence,adu...",en,Waking Life,Waking Life is about a young man in a persiste...,4.3827,"IFC Productions,Thousand Words,Detour Filmprod...",US,2001-10-19,3176880.0,101.0,en,Dreams. What are they? An escape from reality ...,Waking Life,914,7.470
1,3.000000e+06,"Erkan Kolçak Köstendil,Ali Atay,Şebnem Bozoklu...",NR,-1.0,Can Ulkay,"History,Drama,War","world war i,ice cream,gallipoli campaign,broke...",tr,Türk İşi Dondurma,"In 1915, two Turks in Australia make a living ...",1.7656,Dijital Sanatlar,TR,2019-03-15,1289283.0,120.0,"tr,en",NaN,Turkish Ice Cream,42,5.900
2,4.000000e+06,"Dennis Hopper,Asia Argento,Helen Shaver,Lochly...",R,-1.0,Paul Lynch,"Crime,Drama,Thriller","kidnapping,hostage,psychopath,deputy sheriff,s...",en,The Keeper,When an apparently exemplary cop abducts and s...,7.2882,Peace Arch Films,"GB,CA",2004-05-14,73788.0,95.0,en,NaN,The Keeper,32,4.375
3,1.050910e+07,"Jimmy Bennett,Adam Taylor Gordon,Ashley Rose O...",G,-1.0,"Larry Leichliter,Bill Melendez","Animation,Comedy,Family,TV Movie","holiday,dog",en,"I Want a Dog for Christmas, Charlie Brown",Linus and Lucy's younger brother Rerun wants a...,5.2636,"Lee Mendelson Film Productions,Charles M. Schu...",US,2003-12-09,0.0,49.0,en,NaN,"I Want a Dog for Christmas, Charlie Brown",55,5.845
4,6.300000e+05,"John Schneider,Luke Benjamin Bernard,Matthew F...",PG-13,-1.0,Curtis Graham,Drama,Unknown,en,The Favorite,Inspired by the true events of Luke Benjamin B...,3.4839,Unknown,US,2019-09-13,36800000.0,108.0,en,"One brother fights for his life, the other fig...",The Favorite,2,6.000



===== TEST (head) =====


,budget,cast_top5,certification_US,collection_id,directors,genres,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_count,vote_average
0,5.000000e+02,"Erana James,Stig Eldred,Sam Harris",NR,-1.0,Mason Cade Packer,Unknown,Unknown,en,1882,The region is in a state of unrest. Rumors of ...,0.5579,Unknown,Unknown,2017-12-13,0.0,5.0,en,A proof-of-concept short film,1882,1,10.000
1,8.500000e+07,"Ben Schwartz,James Marsden,Tika Sumpter,Jim Ca...",PG,720879.0,Jeff Fowler,"Action,Science Fiction,Comedy,Family","friendship,video game,san francisco, californi...",en,Sonic the Hedgehog,"Powered with incredible speed, Sonic The Hedge...",9.0558,"Original Film,Blur Studio,Marza Animation Plan...","JP,US",2020-02-12,319715683.0,99.0,en,A whole new speed of hero.,Sonic the Hedgehog,10144,7.296
2,1.517000e+08,"Tony Fadil,Elliot Cable,Mark Wingett,Sonera An...",NR,-1.0,Steven M. Smith,"Comedy,Horror,Science Fiction","horror spoof,survival horror,zombie apocalypse",en,Dead Again,A VIRUS IS LOOSE. In rural village where crime...,6.3187,Greenway Entertainment,Unknown,2020-10-16,2000000.0,75.0,en,Lockdown Won't Save You,Dead Again,14,4.700
3,3.558303e+06,"Jesús Bonilla,Santiago Segura,Concha Velasco,A...",NR,-1.0,Jesús Bonilla,"Action,Adventure,Comedy",Unknown,es,El oro de Moscú,In an odd destiny coincidence and hospital emp...,1.3139,Amiguetes Entertainment,ES,2003-03-28,6890947.0,100.0,es,NaN,Moscow Gold,66,5.400
4,5.317828e+06,"Neeru Bajwa,Binnu Dhillon,Anita Devgan,Rana Ra...",NR,-1.0,Pankaj Batra,"Crime,Drama",Unknown,pa,Channo Kamli Yaar Di,A Punjabi girl travels to Canada to find her m...,4.8295,Neeru Bajwa Entertainment,IN,2016-02-19,0.0,144.0,"en,pa",NaN,Channo Kamli Yaar Di,1,4.000



===== TRAIN (info) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                8000 non-null   float64
 1   cast_top5             8000 non-null   object 
 2   certification_US      8000 non-null   object 
 3   collection_id         8000 non-null   float64
 4   directors             8000 non-null   object 
 5   genres                8000 non-null   object 
 6   keywords              8000 non-null   object 
 7   original_language     8000 non-null   object 
 8   original_title        8000 non-null   object 
 9   overview              7932 non-null   object 
 10  popularity            8000 non-null   float64
 11  production_companies  8000 non-null   object 
 12  production_countries  8000 non-null   object 
 13  release_date          8000 non-null   object 
 14  revenue               8000 non-null   float64


# Khai báo hàm cần thiết


In [14]:
# Các hàm tiện ích (nhóm bổ sung/tái sử dụng tại đây)

def prepare_raw_ols_data(
    train_df: pd.DataFrame,
    feature_cols: List[str],
    target_col: str,
) -> Tuple[pd.DataFrame, pd.Series]:
    """Chuẩn bị dữ liệu cho mô hình OLS baseline trên các biến số thô.

    Mô tả:
        - Chỉ dùng train_df để fit OLS (không đụng tới test).
        - Ép các cột đặc trưng về số và điền NaN bằng median để chạy mô hình.
        - Loại bỏ hàng có target NaN (nếu phát sinh).

    Args:
        train_df: DataFrame huấn luyện thô.
        feature_cols: Danh sách cột đặc trưng số thô (ví dụ budget, revenue, popularity, runtime).
        target_col: Tên cột mục tiêu (vote_average).

    Returns:
        (X, y) đã được chuẩn hóa để đưa vào OLS.
    """
    missing_features = [c for c in feature_cols if c not in train_df.columns]
    if len(missing_features) > 0:
        raise ValueError(f"Train thiếu các cột đặc trưng cần cho OLS: {missing_features}")
    if target_col not in train_df.columns:
        raise ValueError(f"Train thiếu cột mục tiêu: {target_col}")

    X = train_df[feature_cols].copy()
    for c in feature_cols:
        X[c] = pd.to_numeric(X[c], errors="coerce")  # Bảo đảm là số để OLS chạy ổn định
        X[c] = X[c].fillna(X[c].median())  # Điền nhanh NaN bằng median (chỉ cho baseline)

    y = pd.to_numeric(train_df[target_col], errors="coerce")
    valid_mask = ~y.isna()
    X = X.loc[valid_mask].copy()
    y = y.loc[valid_mask].copy()
    return X, y


def fit_ols_model(X: pd.DataFrame, y: pd.Series):
    """Fit mô hình OLS và trả về kết quả statsmodels để xem summary().

    Args:
        X: DataFrame đặc trưng (đã xử lý NaN tối thiểu).
        y: Series mục tiêu.

    Returns:
        Đối tượng RegressionResultsWrapper (có phương thức summary()).
    """
    X_const = sm.add_constant(X, has_constant="add")  # Thêm hệ số chặn (intercept)
    return sm.OLS(y, X_const).fit()

# Thí nghiệm sơ bộ (Pilot Experiment): OLS Baseline

## 1) Mục tiêu của bước này
Nhóm xây dựng một baseline tuyến tính (OLS) để định lượng mức độ liên hệ tuyến tính giữa **các biến số thô** và biến mục tiêu `vote_average`. Kết quả dùng làm cơ sở khoa học để quyết định: biến nào giữ nguyên, biến nào cần biến đổi (log/ratio), và biến nào có thể ít giá trị dự báo.

## 2) Cơ sở lý thuyết
Mô hình Ordinary Least Squares (OLS) ước lượng quan hệ tuyến tính:
$$y = \beta_0 + \sum_{j=1}^{p} \beta_j x_j + \varepsilon$$
Trong đó:
- $R^2$ đo mức độ giải thích phương sai của `vote_average` bởi các biến số thô.
- **P-value** (trên kiểm định $H_0: \beta_j = 0$) giúp nhóm đánh giá biến nào có đóng góp tuyến tính có ý nghĩa thống kê (ví dụ p < 0.05).

## 3) Giả thuyết thí nghiệm (Pilot Hypothesis)
Nhóm đặt giả thuyết thí nghiệm rằng ít nhất một trong các biến số thô `budget`, `revenue`, `popularity`, `runtime` có hệ số hồi quy khác 0 một cách có ý nghĩa thống kê, và mô hình OLS đạt $R^2$ lớn hơn mức ngẫu nhiên.

## 4) Thực hiện
- Chọn các biến số thô: `budget`, `revenue`, `popularity`, `runtime`.
- Xử lý nhanh NaN (nếu có) bằng median *chỉ để chạy baseline*.
- Fit OLS: `sm.OLS(y_train, sm.add_constant(X_raw_train)).fit()`.
- Hiển thị `model.summary()`.

## 5) Kết luận sơ bộ
(Sau khi chạy Cell 8, nhóm dừng lại để đọc `R-squared` và các `P>|t|`.)
- Nhóm xác định biến nào có p-value nhỏ (tín hiệu tuyến tính mạnh) và dấu của hệ số (tác động cùng/ ngược chiều).
- Nhóm ghi nhận $R^2$ baseline để làm mốc so sánh sau Feature Engineering.
- Nhóm chưa đưa ra giả thuyết Feature Engineering chính thức ở bước này; các giả thuyết sẽ được chốt sau khi phân tích bảng summary.

In [15]:
# Cell 8 — Chạy OLS baseline để xem tầm quan trọng tuyến tính của biến số thô

raw_numeric_features = ['budget', 'revenue', 'popularity', 'runtime']  # Nhóm biến số thô để kiểm tra baseline
target_col = 'vote_average'  # Biến mục tiêu

X_raw_train, y_train = prepare_raw_ols_data(
    train_df=train_raw,
    feature_cols=raw_numeric_features,
    target_col=target_col,
)  # Chuẩn bị dữ liệu (median-impute nhanh)

print('Số quan sát dùng cho OLS:', len(X_raw_train))
print('Thiếu dữ liệu (sau khi median-impute cho X):')
print(X_raw_train.isna().sum().to_string())

ols_model = fit_ols_model(X_raw_train, y_train)  # Fit OLS (có intercept)
print(ols_model.summary())  # In bảng summary để nhóm phân tích R^2 và P-value

Số quan sát dùng cho OLS: 8000
Thiếu dữ liệu (sau khi median-impute cho X):
budget        0
revenue       0
popularity    0
runtime       0
                            OLS Regression Results                            
Dep. Variable:           vote_average   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     36.14
Date:                Fri, 02 Jan 2026   Prob (F-statistic):           5.57e-30
Time:                        12:58:09   Log-Likelihood:                -14598.
No. Observations:                8000   AIC:                         2.921e+04
Df Residuals:                    7995   BIC:                         2.924e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025    